In [1]:
import pandas               as pd
import numpy                as np
import seaborn              as sns
import plotly.express       as px
import plotly.graph_objects as go
import ipywidgets           as widgets
import geopy.geocoders
from ipywidgets      import fixed, interact, interact_manual
from matplotlib      import pyplot as plt
from geopy.geocoders import Nominatim

# set float type display format
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# set plots size
plt.rcParams['figure.figsize'] = [ 20, 10 ]

### 0. Carga

In [2]:
df_raw = pd.read_csv('../data/processed/kd_house_processed.csv')
print(df_raw.info())
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  int64  
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  int64  
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,year
0,7129300520,2014-10-13,221900.00,3,1,1180,5650,1,0,0,3,7,1180,0,1955,0,98178,47.51,-122.26,2014
1,6414100192,2014-12-09,538000.00,3,2,2570,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.72,-122.32,2014
2,5631500400,2015-02-25,180000.00,2,1,770,10000,1,0,0,3,6,770,0,1933,0,98028,47.74,-122.23,2015
3,2487200875,2014-12-09,604000.00,4,3,1960,5000,1,0,0,5,7,1050,910,1965,0,98136,47.52,-122.39,2014
4,1954400510,2015-02-18,510000.00,3,2,1680,8080,1,0,0,3,8,1680,0,1987,0,98074,47.62,-122.05,2015


### 1. Engenharia de variáveis

In [8]:
#new attributes
data = df_raw.copy()

data['sn_compra'] = 'NA'
data['condition_type'] = 'NA'

data['sell_price'] = 0.0
data['profit'] = 0.0
data['month'] = data['date'].dt.strftime('%m').astype('int64')
data['season'] = data['month'].apply(lambda x: 'winter' if x in(12,1,2) else
                                               'spring' if x in(3,4,5) else
                                               'summer' if x in(6,7,8) else 'fall' )
data['sn_compra'] = data['sn_compra'].astype('string')
data['condition_type'] = data['condition_type'].astype('string')
data['condition_type'] = data['condition'].apply(lambda x: 'good' if x==5 else 'regular' if x in(3,4) else 'bad')
data['season'] = data['season'].astype('string')

# definindo mediana do custo por região
regional_median = data[['zipcode','price']].groupby('zipcode').median().reset_index().copy()
regional_median.columns = ['zipcode','regional_median']
regional_median.to_csv('../data/interim/regional_median.csv',index=False)

# definindo mediana do custo por região/estação
season_median = data[['price','zipcode','season']].groupby(['zipcode','season']).median('price')
season_region_median = season_median.rename(columns={'price': 'season_median'}).reset_index(drop=False)
season_region_median.to_csv('../data/interim/season_region_median.csv',index=False)

**Criando filtros**

In [9]:
f_zipcode = widgets.Dropdown(
    options = data['zipcode'].sort_values().unique().tolist(),
    description = 'Zipcode',
    disable = False
)

f_filters = widgets.Checkbox(
    value=False,
    description='Disable filter',
    disabled=False,
    indent=False
)

**Definição dos imóveis para compra**

In [10]:
# adicionando a mediana regional para cada imóvel
purchase = pd.merge(data,regional_median,how='left',on='zipcode').copy()

purchase['sn_compra'] = purchase.apply(
    lambda x: 'y' if (x['price'] < x['regional_median']) & (x['condition_type']=='good') else 'n',
    axis=1 )

purchase_list = purchase.drop(columns=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront',
                                  'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
                                  'yr_renovated', 'month'])

purchase_list.to_csv('../data/processed/purchase_list.csv',index=False)                                  

**Coleta dos dados de geolocalização dos imóveis elegíveis**

In [ ]:
# dados de geolocalização

geolocation = purchase_list.loc[purchase_list['sn_compra'] == 'y'][['id', 'lat', 'long']].copy().reset_index( drop=True )
geolocator = Nominatim( user_agent='geoapiExercises' )
geolocation['street'] = 'NA'
geolocation['house_num'] = 'NA'
geolocation['city'] = 'NA'
geolocation['neighbourhood'] = 'NA'
geolocation['state'] = 'NA'

for i in range( len( geolocation ) ):
    query = str( geolocation.loc[i, 'lat'] ) + ',' + str( geolocation.loc[i, 'long'] )
    response = geolocator.reverse( query, timeout=10000 )
    response = pd.json_normalize( response.raw['address'] )
    geolocation.iloc[i, 3] = response.apply( lambda x: x['road'] if 'road' in response.columns else 'NA', axis = 1 )    
    geolocation.iloc[i, 4] = response.apply( lambda x: x['house_number'] if 'house_number' in response.columns else 'NA', axis = 1 )
    geolocation.iloc[i, 5] = response.apply( lambda x: x['city'] if 'city' in response.columns else 'NA', axis = 1 )
    geolocation.iloc[i, 6] = response.apply( lambda x: x['neighbourhood'] if 'neighbourhood' in response.columns else
                                                       x['county'] if 'county' in response.columns else 'NA', axis = 1 )
    geolocation.iloc[i, 7] = response.apply( lambda x: x['state'] if 'state' in response.columns else 'NA', axis = 1 )
geolocation['address'] = geolocation['street'] + ', ' + geolocation['house_num']

geolocation.to_csv('../data/processed/geoloc.csv', index=False)

In [11]:
geolocation = pd.read_csv('../data/processed/geoloc.csv')

**Definição do preço de revenda**

In [ ]:
sell = pd.merge(purchase_list,season_region_median,how='inner',on=['zipcode'])
sell = sell.loc[sell['sn_compra'] == 'y']
sell['sell_price'] = sell.apply(lambda x: x['price'] * 1.3 if x['price'] < x['season_median'] else x['price'] * 1.1, axis=1)
sell['diff_price'] = sell.apply(lambda x: np.sqrt((x['sell_price']-x['season_median'])**2),axis=1)

aux = sell[['id','diff_price']].groupby('id').min().reset_index(drop=False)
aux['sn_vende'] = 'y'

sell = pd.merge(sell,aux,how='left',on=['id','diff_price']).drop_duplicates(subset=['id','diff_price'])
sell = sell.loc[sell['sn_vende']=='y']

sell['profit'] = sell['sell_price'] - sell['price']

sell = sell.drop(columns=['date','sn_compra','condition_type','regional_median','season_x','season_median','diff_price','sn_vende'])

sell_list = pd.merge(sell,geolocation[['id','address','neighbourhood','city']],how='left',on='id').rename(columns={'season_y':'season'})

sell_list.to_csv('../data/processed/sell_list.csv',index=False)

In [44]:
# lucro total
sell_list = pd.read_csv('../data/processed/sell_list.csv')
sell_list['profit'].sum()

72079180.8

In [43]:
# top3 regions with more available houses to sell
sell_list[['id','zipcode']].groupby('zipcode').count().sort_values('id',ascending=False).head(1).reset_index()

,zipcode,id
0,98042,49


In [42]:
# top3 most valuable regions
sell_list[['profit','zipcode']].groupby('zipcode').sum().sort_values('profit',ascending=False).head(1).reset_index()

,zipcode,profit
0,98006,6816570.0


In [39]:
# most valuable regions per season
a = sell_list[['profit','zipcode','season']].groupby(['season','zipcode']).sum().sort_values(['season','profit'],ascending=False).reset_index()
b = sell_list[['profit','zipcode','season']].groupby(['season','zipcode']).sum().groupby('season').max().reset_index()
pd.merge(a,b,how='right', on=['season','profit'])

,season,zipcode,profit
0,fall,98006,3100530.0
1,spring,98004,2408160.0
2,summer,98006,3362040.0
3,winter,98117,1405035.0


**Visualização dos imóveis para compra**

In [151]:
def data_viz( df, region ):
    plot = df[df['zipcode']==region][['id','price','zipcode','regional_median','condition_type','sn_compra']]
    plot['color'] = plot['sn_compra'].apply(lambda x: 'lightgray' if x == 'n' else 'blue')
     
    fig = go.Figure()
    
    # Add traces
    fig.add_trace(go.Scatter(y=plot['price'], 
                        marker_color=plot['color'],
                        mode='markers',
                        text='ID: '+plot['id'].astype('string')+' | Region: '+plot['zipcode'].astype('string')+' | Condition: ' + plot['condition_type'] + ' | Price: US$' + plot['price'].astype('string'),
                        name='Imovel'))

    fig.add_trace(go.Scatter(y=plot['regional_median'],
                        mode='lines',
                        text='Region: '+plot['zipcode'].astype('string')+' | Median: US$' + plot['regional_median'].astype('string'),
                        name='Mediana Regional'))
    
    fig.update_layout(
        xaxis_title="Houses",
        yaxis_title="Aquisition cost",
        font=dict( color="#000000" )
    )

    fig.show()
    
    return None

In [152]:
interact(data_viz, df=fixed(purchase_list), region = f_zipcode);

**Visualização dos imóveis para compra/revenda no mapa**

In [162]:
def make_map( house_list, region, filters ): 
    
    map_data = house_list

    if filters:
        map_data
    else:
        map_data = map_data.loc[map_data['zipcode']==region]
    
    _map = px.scatter_mapbox( map_data,
                              lat='lat',
                              lon='long',
                              hover_name='address',                              
                              color='season',
                              color_discrete_sequence=px.colors.qualitative.Dark2,
                              zoom=10,
                              text='ID: '+map_data['id'].astype('string'),
                              hover_data=dict( lat=False, long=False, season=False)
                            )

    _map.update_layout(mapbox_style='open-street-map')
    _map.update_layout(height=600, margin = {'r':0,'t':0,'l':0,'b':0})
    _map.show()
    
    return None

In [163]:
interact( make_map, house_list=fixed( sell_list ), region=f_zipcode, filters=f_filters );